In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import  ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_length=20,
    max_token_limit=100,
    return_messages=True,

)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "Titanic",
        "answer": """
       🚢💑🧊
        """,
    },
    {
        "movie": "Inception",
        "answer": """
       🧠🌀🕰️
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]



example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert who explains the movie with three emojis."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "What do you wnant to express about {movie}?"),
    ]
)

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)



In [3]:
invoke_chain("Inception")

🧠🌀🕰️content='🧠🌀🕰️'


In [4]:
invoke_chain("Jurassic Park")

🦖🌴🔬content='🦖🌴🔬'


In [5]:
invoke_chain("Frozen") 

❄️👸⛄content='❄️👸⛄'


In [7]:
invoke_chain("Jaws")

🦈🏖️🚣‍♂️The human mentions "Inception" and the AI responds with emojis representing the complexity and time-bending nature of the movie.content='🦈🏖️🚣\u200d♂️'


In [8]:
invoke_chain("Star Wars") 

🌌⚔️🤖The human mentions "Inception" and the AI responds with emojis representing the complexity and time-bending nature of the movie. When the human brings up "Jurassic Park," the AI responds with emojis of a dinosaur, palm tree, and microscope.content='🌌⚔️🤖'


In [9]:
invoke_chain("King Kong")

🦍🏢🌆The human mentions "Inception" and the AI responds with emojis representing the complexity and time-bending nature of the movie. When the human brings up "Jurassic Park," the AI responds with emojis of a dinosaur, palm tree, and microscope. In response to "Frozen," the AI uses emojis of snowflake, princess, and snowman.content='🦍🏢🌆'


In [ ]:
invoke_chain("The Dark Knight")